<a href="https://colab.research.google.com/github/kfeng2/kftesting/blob/main/r_drop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes


In [ ]:
!kaggle datasets download -d yasufuminakama/cpc-data
!kaggle datasets download -d yasufuminakama/pppm-pip-wheels-dataset
!kaggle competitions download -c us-patent-phrase-to-phrase-matching
! mkdir pppm-pip-wheels-dataset
! unzip pppm-pip-wheels-dataset.zip -d pppm-pip-wheels-dataset
! mkdir us-patent-phrase-to-phrase-matching
! unzip us-patent-phrase-to-phrase-matching.zip -d us-patent-phrase-to-phrase-matching
! mkdir cpc-data
! unzip cpc-data.zip -d cpc-data
!pip download transformers==4.18.0
!pip download tokenizers==0.12.1
!pip install wandb -qqq

 62% 9.00M/14.4M [00:00<00:00, 17.7MB/s]
100% 14.4M/14.4M [00:00<00:00, 28.5MB/s]
 98% 33.0M/33.5M [00:00<00:00, 62.1MB/s]
100% 33.5M/33.5M [00:00<00:00, 45.0MB/s]
  0% 0.00/682k [00:00<?, ?B/s]
100% 682k/682k [00:00<00:00, 113MB/s]
Archive:  pppm-pip-wheels-dataset.zip
  inflating: pppm-pip-wheels-dataset/PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl  
  inflating: pppm-pip-wheels-dataset/certifi-2021.10.8-py2.py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/charset_normalizer-2.0.12-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/click-8.0.4-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/filelock-3.6.0-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/huggingface_hub-0.4.0-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/idna-3.3-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/importlib_metadata-4.11.3-py3-none-any.whl  
  inflating: pppm-pip-wheels-dataset/joblib-1.1.0-py2.py3-

In [ ]:
import os
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=./ transformers')
os.system('python -m pip install --no-index --find-links=./ tokenizers')
os.system('python -m pip install --no-index --find-links=./pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=./pppm-pip-wheels-dataset tokenizers')
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.1 MB/s 


In [ ]:
#import all libraries when restart runtime

import os
import wandb
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

print(f"torch.__version__: {torch.__version__}")
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")


%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [ ]:
#connect google drive
from google.colab import drive
! mkdir gdrive
drive.mount('../content/gdrive')

mkdir: cannot create directory ‘gdrive’: File exists
Drive already mounted at ../content/gdrive; to attempt to forcibly remount, call drive.mount("../content/gdrive", force_remount=True).


In [ ]:

INPUT_DIR = '../content/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
#model config

class CFG:
    wandb=True
    competition='PPPM'
    #_wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=500
    epochs = 4
    encoder_lr=2e-5
    decoder_lr=2e-5
    #min_lr=5e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size = 16
    fc_dropout=0.2
    target_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=55
    n_fold = 5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
if CFG.wandb:
    
    #import wandb

    try:
        
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('Didnt log in yet')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='PPPM-Public-over', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

wandb: Currently logged in as: kfeng2. Use `wandb login --relogin` to force relogin


In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

def get_logger(filename=OUTPUT_DIR+'/gdrive/MyDrive/train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed = CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything()

In [ ]:
#Load data
train = pd.read_csv(INPUT_DIR+'train.csv')
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"train.shape: {train.shape}")
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")

train.shape: (36473, 5)
test.shape: (36, 4)
submission.shape: (36, 2)


In [ ]:
# rewrite cpc dict, make it clear and concise

mod_cpc = {'A01': 'HUMAN NECESSITIES. agriculture forestry animal husbandry',
 'A21': 'HUMAN NECESSITIES. baking edible doughs',
 'A22': 'HUMAN NECESSITIES. butchering meat treatment processing poultry or fish',
 'A23': 'HUMAN NECESSITIES. foods or foodstuffs treatment thereof',
 'A24': "HUMAN NECESSITIES. tobacco cigars cigarettes",
 'A41': 'HUMAN NECESSITIES. wearing apparel',
 'A42': 'HUMAN NECESSITIES. headwear',
 'A43': 'HUMAN NECESSITIES. footwear',
 'A44': 'HUMAN NECESSITIES. haberdashery jewellery',
 'A45': 'HUMAN NECESSITIES. hand or travelling articles',
 'A46': 'HUMAN NECESSITIES. brushware',
 'A47': 'HUMAN NECESSITIES. furniture domestic articles spice mills suction cleaners',
 'A61': 'HUMAN NECESSITIES. medical or veterinary science  hygiene',
 'A62': 'HUMAN NECESSITIES. life saving fire fighting',
 'A63': 'HUMAN NECESSITIES. sports games amusements',
 'A99': 'HUMAN NECESSITIES. others',
 'B01': 'PERFORMING OPERATIONS  TRANSPORTING. physical or chemical processes or apparatus',
 'B02': 'PERFORMING OPERATIONS  TRANSPORTING. crushing pulverising or disintegrating milling',
 'B03': 'PERFORMING OPERATIONS  TRANSPORTING. separation of solid materials using liquids or electricity',
 'B04': 'PERFORMING OPERATIONS  TRANSPORTING. centrifugal apparatus or machines for carrying out chemical',
 'B05': 'PERFORMING OPERATIONS  TRANSPORTING. spraying or atomising applying fluent materials to surfaces',
 'B06': 'PERFORMING OPERATIONS  TRANSPORTING. generating or transmitting mechanical vibrations',
 'B07': 'PERFORMING OPERATIONS  TRANSPORTING. separating solids from solids sorting',
 'B08': 'PERFORMING OPERATIONS  TRANSPORTING. cleaning',
 'B09': 'PERFORMING OPERATIONS  TRANSPORTING. disposal of solid waste reclamation of contaminated soil',
 'B21': 'PERFORMING OPERATIONS  TRANSPORTING. mechanical metal working without removing material punching metal',
 'B22': 'PERFORMING OPERATIONS  TRANSPORTING. casting powder metallurgy',
 'B23': 'PERFORMING OPERATIONS  TRANSPORTING. machine tools metal working ',
 'B24': 'PERFORMING OPERATIONS  TRANSPORTING. grinding polishing',
 'B25': 'PERFORMING OPERATIONS  TRANSPORTING. hand tools portable power driven tools manipulators',
 'B26': 'PERFORMING OPERATIONS  TRANSPORTING. hand cutting tools cutting severing',
 'B27': 'PERFORMING OPERATIONS  TRANSPORTING. working or preserving wood nailing stapling',
 'B28': 'PERFORMING OPERATIONS  TRANSPORTING. working cement clay stone',
 'B29': 'PERFORMING OPERATIONS  TRANSPORTING. working of plastics',
 'B30': 'PERFORMING OPERATIONS  TRANSPORTING. presses',
 'B31': 'PERFORMING OPERATIONS  TRANSPORTING. making articles of paper cardboard paper',
 'B32': 'PERFORMING OPERATIONS  TRANSPORTING. layered products',
 'B33': 'PERFORMING OPERATIONS  TRANSPORTING. additive manufacturing technology',
 'B41': 'PERFORMING OPERATIONS  TRANSPORTING. printing  lining machines typewriters stamps',
 'B42': 'PERFORMING OPERATIONS  TRANSPORTING. bookbinding albums files special printed matter',
 'B43': 'PERFORMING OPERATIONS  TRANSPORTING. writing or drawing implements bureau accessories',
 'B44': 'PERFORMING OPERATIONS  TRANSPORTING. decorative arts',
 'B60': 'PERFORMING OPERATIONS  TRANSPORTING. vehicles',
 'B61': 'PERFORMING OPERATIONS  TRANSPORTING. railways',
 'B62': 'PERFORMING OPERATIONS  TRANSPORTING. land vehicles for travelling ',
 'B63': 'PERFORMING OPERATIONS  TRANSPORTING. ships or other waterborne vessels  related equipment',
 'B64': 'PERFORMING OPERATIONS  TRANSPORTING. aircraft aviation cosmonautics',
 'B65': 'PERFORMING OPERATIONS  TRANSPORTING. conveying packing storing ',
 'B66': 'PERFORMING OPERATIONS  TRANSPORTING. hoisting lifting  hauling',
 'B67': 'PERFORMING OPERATIONS  TRANSPORTING. bottles jars containers liquid handling',
 'B68': 'PERFORMING OPERATIONS  TRANSPORTING. saddlery upholstery',
 'B81': 'PERFORMING OPERATIONS  TRANSPORTING. microstructural technology',
 'B82': 'PERFORMING OPERATIONS  TRANSPORTING. nanotechnology',
 'B99': 'PERFORMING OPERATIONS  TRANSPORTING. others',
 'C01': 'CHEMISTRY  METALLURGY. inorganic chemistry',
 'C02': 'CHEMISTRY  METALLURGY. water waste water sewage or sludge',
 'C03': 'CHEMISTRY  METALLURGY. glass mineral slag wool',
 'C04': 'CHEMISTRY  METALLURGY. cements concrete artificial stone ceramics refractories',
 'C05': 'CHEMISTRY  METALLURGY. fertilisers  manufacture thereof',
 'C06': 'CHEMISTRY  METALLURGY. explosives  matches',
 'C07': 'CHEMISTRY  METALLURGY. organic chemistry',
 'C08': 'CHEMISTRY  METALLURGY. organic macromolecular compounds',
 'C09': 'CHEMISTRY  METALLURGY. dyes paints polishes natural resins adhesives compositions ',
 'C10': 'CHEMISTRY  METALLURGY. petroleum gas or coke containing carbon monoxide fuels lubricants peat',
 'C11': 'CHEMISTRY  METALLURGY. oils fats waxes fatty acids therefrom detergents candles',
 'C12': 'CHEMISTRY  METALLURGY. biochemistry microbiology enzymology mutation genetic engineering',
 'C13': 'CHEMISTRY  METALLURGY. sugar industry',
 'C14': 'CHEMISTRY  METALLURGY. skins hides pelts leather',
 'C21': 'CHEMISTRY  METALLURGY. metallurgy of iron',
 'C22': 'CHEMISTRY  METALLURGY. metallurgy ferrous or alloys treatment',
 'C23': 'CHEMISTRY  METALLURGY. coating metallic chemical surface vacuum evaporation sputtering',
 'C25': 'CHEMISTRY  METALLURGY. electrolytic or electrophoretic processes  apparatus therefor',
 'C30': 'CHEMISTRY  METALLURGY. crystal growth',
 'C40': 'CHEMISTRY  METALLURGY. combinatorial technology',
 'C99': 'CHEMISTRY  METALLURGY. others',
 'D01': 'TEXTILES  PAPER. threads or fibres spinning',
 'D02': 'TEXTILES  PAPER. yarns or ropes warping or beaming',
 'D03': 'TEXTILES  PAPER. weaving',
 'D04': 'TEXTILES  PAPER. braiding lace making knitting trimmings fabrics',
 'D05': 'TEXTILES  PAPER. sewing embroidering tufting',
 'D06': 'TEXTILES  PAPER. textiles or the like laundering flexible materials',
 'D07': 'TEXTILES  PAPER. ropes cables other than electric',
 'D10': 'TEXTILES  PAPER. sublasses of textiles',
 'D21': 'TEXTILES  PAPER. paper making production of cellulose',
 'D99': 'TEXTILES  PAPER. others',
 'E01': 'FIXED CONSTRUCTIONS. roads railways or bridges',
 'E02': 'FIXED CONSTRUCTIONS. hydraulic engineering foundations soil shifting',
 'E03': 'FIXED CONSTRUCTIONS. water supply sewerage',
 'E04': 'FIXED CONSTRUCTIONS. building',
 'E05': 'FIXED CONSTRUCTIONS. locks keys window or door fittings  safes',
 'E06': 'FIXED CONSTRUCTIONS. doors windows shutters  or roller blinds  ladders',
 'E21': 'FIXED CONSTRUCTIONS. earth drilling mining',
 'E99': 'FIXED CONSTRUCTIONS. others',
 'F01': 'MECHANICAL ENGINEERING. engine plants steam engines',
 'F02': 'MECHANICAL ENGINEERING. combustion engines hot gas ',
 'F03': 'MECHANICAL ENGINEERING. liquids wind spring or weight motors',
 'F04': 'MECHANICAL ENGINEERING. positive displacement for liquids pumps for elastic fluids',
 'F05': 'MECHANICAL ENGINEERING. indexing schemes relating to engines or pumps',
 'F15': 'MECHANICAL ENGINEERING. fluid pressure actuators  hydraulics or pneumatics ',
 'F16': 'MECHANICAL ENGINEERING. engineering elements units or installations thermal insulation',
 'F17': 'MECHANICAL ENGINEERING. storing or distributing gases or liquids',
 'F21': 'MECHANICAL ENGINEERING. lighting',
 'F22': 'MECHANICAL ENGINEERING. steam generation',
 'F23': 'MECHANICAL ENGINEERING. combustion',
 'F24': 'MECHANICAL ENGINEERING. heating ranges ventilating',
 'F25': 'MECHANICAL ENGINEERING. cooling refrigeration liquefaction solidification',
 'F26': 'MECHANICAL ENGINEERING. drying',
 'F27': 'MECHANICAL ENGINEERING. furnaces kilns ovens retorts',
 'F28': 'MECHANICAL ENGINEERING. heat exchange ',
 'F41': 'MECHANICAL ENGINEERING. weapons',
 'F42': 'MECHANICAL ENGINEERING. ammunition  blasting',
 'F99': 'MECHANICAL ENGINEERING. others',
 'G01': 'PHYSICS. measuring  testing',
 'G02': 'PHYSICS. optics',
 'G03': 'PHYSICS. photography cinematography electrography holography',
 'G04': 'PHYSICS. horology',
 'G05': 'PHYSICS. controlling  regulating',
 'G06': 'PHYSICS. computing  calculating  counting',
 'G07': 'PHYSICS. checking devices',
 'G08': 'PHYSICS. signalling',
 'G09': 'PHYSICS. education cryptography display advertising seals',
 'G10': 'PHYSICS. musical instruments  acoustics',
 'G11': 'PHYSICS. information storage',
 'G12': 'PHYSICS. instrument details',
 'G16': 'PHYSICS. information communication technology ',
 'G21': 'PHYSICS. nuclear physics nuclear engineering',
 'G99': 'PHYSICS. others',
 'H01': 'ELECTRICITY. basic electric elements',
 'H02': 'ELECTRICITY. generation distribution of power',
 'H03': 'ELECTRICITY. basic electronic circuitry',
 'H04': 'ELECTRICITY. electric communication',
 'H05': 'ELECTRICITY. electric techniques ',
 'H99': 'ELECTRICITY. others',
 'Y02': 'CROSS SECTIONAL TECHNOLOGIES. adaptation against climate change',
 'Y04': 'CROSS SECTIONAL TECHNOLOGIES. information or communication technologies',
 'Y10': 'CROSS SECTIONAL TECHNOLOGIES. technical subjects covered by former united states patent'}

In [ ]:
#cpc function used before, but didn't use after rewrite the dict
def remove_spchar(string):
    re_string = string.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
    return re_string


def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('../content/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'../content/cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = remove_spchar(result[0].lstrip(pattern))
        
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            r_trans = remove_spchar(result[0].lstrip(pattern))
            results[context] = cpc_result.lower() + ". " + r_trans.lower()
    return results



In [ ]:
cpc_texts = mod_cpc
torch.save(cpc_texts, OUTPUT_DIR+f"/gdrive/MyDrive/cpc_texts{CFG.seed}.pth")
train['context_text'] = train['context'].map(cpc_texts)
test['context_text'] = test['context'].map(cpc_texts)


In [ ]:
train['text'] =train['target'] +'[SEP]' + train['anchor'] + ';'  + train['context_text']+'[SEP]' + train['anchor']
test['text'] = test['target'] + '[SEP]' +test['anchor'] +  ';'  + test['context_text']+ '[SEP]' +test['anchor']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold

def sep_context_group(train, group_list):
    return train[train['context'].apply(lambda x: x[0]).isin(group_list)].reset_index(drop=True)

def cv_split(train):
    train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
    encoder = LabelEncoder()
    train['anchor_map'] = encoder.fit_transform(train['anchor'])
    
    Fold = StratifiedGroupKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
    for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'], groups=train['anchor_map'])):
        train.loc[val_index, 'fold'] = int(n)
    train['fold'] = train['fold'].astype(int)
    display(train.groupby('fold').size())
    return train

In [ ]:
train = cv_split(train)

fold
0    7338
1    7167
2    7470
3    7422
4    7076
dtype: int64

In [ ]:
#tokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR+'/gdrive/MyDrive/tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# ====================================================
# Define max_len
# ====================================================
lengths_dict = {}

lengths = []
tk0 = tqdm(cpc_texts.values(), total=len(cpc_texts))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
lengths_dict['context_text'] = lengths

for text_col in ['anchor', 'target']:
    lengths = []
    tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths
    
CFG.max_len = max(lengths_dict['anchor']) + max(lengths_dict['anchor']) + max(lengths_dict['target'])\
                + max(lengths_dict['context_text']) + 5 # CLS + SEP + SEP + SEP
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/36473 [00:00<?, ?it/s]

  0%|          | 0/36473 [00:00<?, ?it/s]

max_len: 73


In [ ]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['score'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label

In [ ]:
class PhraseDataset:
    def __init__(self, anchor, target, context, score, tokenizer, max_len):
        self.anchor = anchor
        self.target = target
        self.context = context
        self.score = score
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.anchor)

    def __getitem__(self, item):
        anchor = self.anchor[item]
        context = self.context[item]
        target = self.target[item]
        score = self.score[item]

        encoded_text = CFG.tokenizer.encode_plus(
            context + " " + anchor,
            target,
            padding="max_length",
            max_length=self.max_len,
            truncation=True,
        )
        input_ids = encoded_text["input_ids"]
        attention_mask = encoded_text["attention_mask"]

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "score": torch.tensor(score, dtype=torch.float),
        }

In [ ]:
class PatentModel(torch.nn.Module):
    def __init__(self):
        super(PatentModel, self).__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(CFG.model)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        
        self.model = AutoModel.from_pretrained(CFG.model, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(config.hidden_size, CFG.target_size)
        
    def forward(self, ids, mask):
        transformer_out = self.model(input_ids=ids, attention_mask=mask)
        last_hidden_states = transformer_out[0]
        last_hidden_states = self.dropout(torch.mean(last_hidden_states, 1))
        logits1 = self.output(self.dropout1(last_hidden_states))
        logits2 = self.output(self.dropout2(last_hidden_states))
        logits3 = self.output(self.dropout3(last_hidden_states))
        logits4 = self.output(self.dropout4(last_hidden_states))
        logits5 = self.output(self.dropout5(last_hidden_states))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        logits = self.output(last_hidden_states)
        return logits

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout_0 = nn.Dropout(cfg.fc_dropout)
        self.fc_dropout_1 = nn.Dropout(cfg.fc_dropout+0.1)
        self.fc_dropout_2 = nn.Dropout(cfg.fc_dropout+0.2)
        self.fc_dropout_3 = nn.Dropout(cfg.fc_dropout+0.3)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)


    def feature(self, inputs):
         outputs = self.model(**inputs)
         last_hidden_states = outputs[0]
         # feature = torch.mean(last_hidden_states, 1)
         weights = self.attention(last_hidden_states)
         feature = torch.sum(weights * last_hidden_states, dim=1)
         return feature

    def forward(self, inputs):
         feature = self.feature(inputs) 
         output_0 = self.fc(self.fc_dropout_0(feature))
         output_1 = self.fc(self.fc_dropout_1(feature))
         output_2 = self.fc(self.fc_dropout_2(feature))
         output_3 = self.fc(self.fc_dropout_3(feature))
         output = torch.mean(torch.stack((output_0,output_1,output_2,output_3)),0)
       #  output = torch.mean((output_0,output_1,output_2,output_3))
         return output

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

#R-drop function
def compute_kl_loss(p, q, pad_mask=None):

    p_loss = F.kl_div(F.log_softmax(p, dim=-1), F.softmax(q, dim=-1), reduction='none')
    q_loss = F.kl_div(F.log_softmax(q, dim=-1), F.softmax(p, dim=-1), reduction='none')

    if pad_mask is not None:
        p_loss.masked_fill_(pad_mask, 0.)
        q_loss.masked_fill_(pad_mask, 0.)

    p_loss = p_loss.mean()
    q_loss = q_loss.mean()

    loss = (p_loss + q_loss) / 2
    return loss

def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    # for step, (inputs, labels) in enumerate(valid_loader):
    #     for k, v in inputs.items():
    #         inputs[k] = v.to(device)
    #     labels = labels.to(device)
    #     batch_size = labels.size(0)
    for step, data in enumerate(train_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['score'].to(device, dtype = torch.float)
        batch_size = targets.size(0)
       # print(f'step: {step}, inputs: {inputs} , labels: {labels} ')
        with torch.cuda.amp.autocast(enabled=CFG.apex):
          #pass twice to use r-drop
            y_preds = model(ids, mask)
            y_preds_2 = model(ids, mask)
            ce_loss = 0.5 * (criterion(y_preds.view(-1, 1), targets.view(-1, 1)) + criterion(y_preds_2.view(-1, 1), targets.view(-1, 1)))
            kl_loss = compute_kl_loss( y_preds, y_preds_2)
            loss = ce_loss + 0.7 * kl_loss

       # loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.6f}({loss.avg:.6f}) '
                  'Grad: {grad_norm:.6f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, data in enumerate(valid_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['score'].to(device, dtype = torch.float)
        batch_size = targets.size(0)
    # for step, (inputs, labels) in enumerate(valid_loader):
    #     for k, v in inputs.items():
    #         inputs[k] = v.to(device)
    #     labels = labels.to(device)
    #     batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(ids, mask)
        loss = criterion(y_preds.view(-1, 1), targets.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.6f}({loss.avg:.6f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    predictions = np.concatenate(predictions)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
class PatentModel(torch.nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super(PatentModel, self).__init__()
        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        #self.config = AutoConfig.from_pretrained(CFG.model)

        self.config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        
        #self.model = AutoModel.from_pretrained(CFG.model, config=self.config)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(self.config.hidden_size, CFG.target_size)
        
    def forward(self, ids, mask):
        transformer_out = self.model(input_ids=ids, attention_mask=mask)
        last_hidden_states = transformer_out[0]
        last_hidden_states = self.dropout(torch.mean(last_hidden_states, 1))
        logits1 = self.output(self.dropout1(last_hidden_states))
        logits2 = self.output(self.dropout2(last_hidden_states))
        logits3 = self.output(self.dropout3(last_hidden_states))
        logits4 = self.output(self.dropout4(last_hidden_states))
        logits5 = self.output(self.dropout5(last_hidden_states))
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        logits = self.output(last_hidden_states)
        return logits

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds['score'].values

    train_dataset = PhraseDataset(
        train_folds.anchor.values,
        train_folds.target.values,
        train_folds.context_text.values,
        train_folds.score.values,
        CFG.tokenizer, 
        CFG.max_len
    ) 
    
    valid_dataset = PhraseDataset(
        valid_folds.anchor.values,
        valid_folds.target.values,
        valid_folds.context_text.values,
        valid_folds.score.values,
        CFG.tokenizer, 
        CFG.max_len
    ) 
    #train_dataset = TrainDataset(CFG, train_folds)
    #valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    model = PatentModel(CFG, config_path=None, pretrained=True)
    #model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+f'/gdrive/MyDrive/config{CFG.seed}.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

#try different loss fn
    #criterion = nn.BCEWithLogitsLoss(reduction="mean")
    criterion = nn.MSELoss(reduction="mean")
    
    best_score = 0.
    ign_epoch = 0
    savebest_valloss = []
    for epoch in range(CFG.epochs):
        #early stop 
        if ign_epoch > 1:
            LOGGER.info('Skip the following epoch')
        else:
            start_time = time.time()

            avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)
            avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)

            score = get_score(valid_labels, predictions)
            savebest_valloss.append(avg_val_loss)
            elapsed = time.time() - start_time

            LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_val_loss: {avg_val_loss:.6f}  time: {elapsed:.0f}s')
            LOGGER.info(f'Epoch {epoch+1} - Score: {score:.6f}')
            if CFG.wandb:
                wandb.log({f"[fold{fold}] epoch": epoch+1, 
                           f"[fold{fold}] avg_train_loss": avg_loss, 
                           f"[fold{fold}] avg_val_loss": avg_val_loss,
                           f"[fold{fold}] score": score})

            if best_score < score:
                best_score = score
                LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.6f} Model ')
                torch.save({'model': model.state_dict(),
                            'predictions': predictions},
                            OUTPUT_DIR+f"/gdrive/MyDrive/{CFG.model.replace('/', '-')}_fold{fold}_best{CFG.seed}.pth")
            if score < 0.75:
                ign_epoch+=2
            if best_score > (score + 0.001) and avg_val_loss> min(savebest_valloss) :
                ign_epoch+=2

    predictions = torch.load(OUTPUT_DIR+f"/gdrive/MyDrive/{CFG.model.replace('/', '-')}_fold{fold}_best{CFG.seed}.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds['pred'] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df['score'].values
        preds = oof_df['pred'].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.6f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in [3,4,5]:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+f'/gdrive/MyDrive/oof_df{CFG.seed}.pkl')
        
    if CFG.wandb:
        wandb.finish()

========== fold: 3 training ==========
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Epoch: [1][0/1815] Elapsed 0m 1s (remain 42m 57s) Loss: 0.460739(0.460739) Grad: inf  LR: 0.00000004  
Epoch: [1][100/1815] Elapsed 1m 49s (remain 30m 59s) Loss: 0.078269(0.245151) Grad: 9594.772461  LR: 0.00000404  
Epoch: [1][200/1815] Elapsed 3m 37s (remain 29m 9s) Loss: 0.046868(0.155339) Grad: 7977.028320  LR: 0.00000804  
